### Create a shapefile with areas that burned at least one time

In [ ]:
burn_area_folder = '/home/jasp/mystuff/cwsig/ex2_fire/aardidas/'

refrst = '/home/jasp/mystuff/cwsig/ex2_fire/topografia/dem_pnse.tif'

In [ ]:
import os

from glass.pys.oss import lst_ff, fprop
from glass.wenv.grs import run_grass

In [ ]:
# List burn areas shapes
bshps = lst_ff(burn_area_folder, file_format='.shp')

In [ ]:
# Start GRASS GIS Session

ws = os.path.dirname(burn_rst)
loc = f'locprod_{orst_name}'

gb = run_grass(ws, location=loc, srs=refrst)

import grass.script.setup as gsetup

gsetup.init(gb, ws, loc, 'PERMANENT')

In [ ]:
from glass.it.shp import shp_to_grs
from glass.gp.ovl.import grsunion
from glass.tbl.grs import reset_table
from glass.gp.gen  import dissolve

In [ ]:
res = None
for shp in bshps:
    # Import shapes into grass
    bshp = shp_to_grs(shp, fprop(shp, 'fn'))
    
    # Union
    if not res:
        res = bshp
    else:
        res = grsunion(res, bshp, f'un_{bshp}')
    
    # Reset table
    reset_table(res, {'code' : 'integer'}, {'code' : '1'})
    
    # Dissolve
    res = dissolve(res, f'ds_{bshp}', 'code', api='pygrass')